In [1]:
import os
from dotenv import load_dotenv
from src.components.ingestion import IngestionComponent
from src.components.transformer import TransformerComponent
from src.components.trainer import TrainerComponent
from src.components.tuner import TunerComponent
from src.config.config import TRAIN_DATA

c:\Users\ernes\Documents\ML Projects\brain_cancer\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
_=load_dotenv()

In [3]:
ingestion=IngestionComponent()
train_path, valid_path, test_path=ingestion.init_component()

In [4]:
transformer=TransformerComponent()
train_dataset,valid_dataset,test_dataset=transformer.init_transform(train_path=train_path, valid_path=valid_path, test_path=test_path)

Found 5613 files belonging to 4 classes.
Found 697 files belonging to 4 classes.
Found 697 files belonging to 4 classes.


In [26]:
tuner=TunerComponent()
hp=tuner.init_tuner(train_dataset=train_dataset, test_dataset=valid_dataset, epochs=2,max_trials=2)

Trial 2 Complete [00h 09m 08s]
val_loss: 0.21958307921886444

Best val_loss So Far: 0.21958307921886444
Total elapsed time: 00h 19m 54s


In [27]:
hp

In [28]:
hp=hp.get_best_hyperparameters(1)[0]
hp

In [5]:
hp={
    'num_con_layers':3,
    'activation_fn': 'selu',
    'units':128,
    'lr':0.0002557125884013172
}

In [32]:
trainer=TrainerComponent()
model_path, model_id, hist = trainer.init_trainer(train_dataset=train_dataset, test_dataset=valid_dataset, epochs=2, hp=hp)

Epoch 1/2
176/176 ━━━━━━━━━━━━━━━━━━━━ 279s 2s/step - accuracy: 0.4253 - loss: 74.3925 - val_accuracy: 0.2568 - val_loss: 13.0251
Epoch 2/2
176/176 ━━━━━━━━━━━━━━━━━━━━ 277s 2s/step - accuracy: 0.6760 - loss: 0.9292 - val_accuracy: 0.4835 - val_loss: 5.1408


In [36]:
model_path, model_id, hist

('c:\\Users\\ernes\\Documents\\ML Projects\\brain_cancer\\artifacts\\models\\1730228715.8542323',
 1730228715.8542323,
 <keras.src.callbacks.history.History at 0x1abea825b50>)

In [37]:
import tensorflow as tf

imported=tf.saved_model.load(model_path)
f=imported.signatures["serving_default"]

In [38]:
for example, label in test_dataset.take(10):
    print(f(example)['output_0'].numpy().argmax(-1))
    print('labels', label)

[0 3 0 0 0 3 0 2 3 3 3 3 2 3 0 1 3 1 0 0 0 0 0 3 3 0 0 0 3 1 1 3]
labels tf.Tensor([1 1 2 2 0 1 2 1 3 2 3 1 2 3 1 1 1 1 2 0 2 2 3 3 3 2 2 2 2 1 1 1], shape=(32,), dtype=int32)
[0 3 3 0 3 0 0 3 0 3 3 0 0 2 0 3 3 0 0 2 1 0 0 1 3 0 3 3 0 3 0 3]
labels tf.Tensor([1 1 3 0 3 0 1 3 0 2 3 3 2 1 0 3 3 1 0 1 1 0 2 1 3 1 3 1 0 1 2 1], shape=(32,), dtype=int32)
[0 3 0 3 2 3 3 0 0 3 0 3 3 0 0 3 3 0 3 3 3 0 3 3 0 0 0 2 3 2 0 0]
labels tf.Tensor([0 3 3 3 2 3 3 2 2 3 0 2 3 1 0 1 1 0 1 3 2 0 3 3 0 0 0 2 3 0 0 0], shape=(32,), dtype=int32)
[0 0 2 0 3 0 1 3 0 0 3 0 3 0 0 3 3 0 0 0 0 0 0 0 0 0 0 2 0 3 0 0]
labels tf.Tensor([2 3 2 1 2 1 1 1 0 0 3 0 3 0 0 3 3 1 2 0 1 0 0 3 3 2 2 2 2 2 3 1], shape=(32,), dtype=int32)
[3 3 3 0 1 1 3 0 0 3 0 0 3 0 3 3 3 0 0 0 0 0 0 0 3 0 0 3 0 3 0 0]
labels tf.Tensor([1 3 3 0 1 1 2 2 0 3 1 2 3 2 1 3 1 0 1 0 2 0 0 2 1 2 1 1 2 1 0 0], shape=(32,), dtype=int32)
[3 0 0 3 0 0 0 3 0 3 0 0 0 0 0 0 3 3 3 2 3 3 0 3 3 1 0 0 3 0 0 0]
labels tf.Tensor([3 0 2 3 2 0 0 1 2 3 2 1 1 1 1 2 3 2 